# Stencil F90 MPI

In [8]:
%%writefile stencil_mpi.f90
program stencil
    use MPI
    implicit none
    integer :: n=0          ! nxn grid
    integer :: energy=0     ! energy to be injected per iteration
    integer :: niters=0     ! number of iterations
    integer :: iters, i, j, px, py, rx, ry
    integer :: north, south, west, east, bx, by, offx, offy
    integer :: nargs=0, iargc
    integer :: mpirank, mpisize, mpitag=1, mpierror
    integer, dimension(3) :: args
    integer, dimension(2) :: pdims=0
    integer, dimension(4) :: sendrequest, recvrequest
    double precision :: mpiwtime=0.0, heat=0.0, rheat=0.0
    double precision, dimension(:), allocatable   :: sendnorthgz, sendsouthgz
    double precision, dimension(:), allocatable   :: recvnorthgz, recvsouthgz
    double precision, dimension(:,:), allocatable :: aold, anew
    character(len=50)                             :: argv

    integer, parameter  :: nsources=3        ! three heat sources
    ! locnsources = number of sources in my area
    integer             :: locnsources=0, locx, locy
    ! locsources = sources local to my rank
    integer, dimension(nsources, 2) :: locsources=0, sources

    call MPI_Init(mpierror)
    call MPI_Comm_rank(MPI_COMM_WORLD, mpirank, mpierror)
    call MPI_Comm_size(MPI_COMM_WORLD, mpisize, mpierror)

    if (mpirank == 0) then                          ! rank 0 argument checking
        mpiwtime = -MPI_Wtime()     ! inicializa contador de tempo
        nargs = iargc()
        call getarg(1, argv); read(argv, *) n       ! nxn grid
        call getarg(2, argv); read(argv, *) energy  ! energy to be injected
        call getarg(3, argv); read(argv, *) niters  ! number of iterations
        args = [ n, energy, niters ]                ! distribute arguments
        call MPI_Bcast(args, 3, MPI_INTEGER, 0, MPI_COMM_WORLD, mpierror)
    else
        call MPI_Bcast(args, 3, MPI_INTEGER, 0, MPI_COMM_WORLD, mpierror)
        n = args(1); energy = args(2); niters  = args(3)
    endif
    
    ! Creates a division of processors in a Cartesian grid
    ! MPI_DIMS_CREATE(NNODES, NDIMS, DIMS, IERROR)
    !   NNODES - number of nodes in a grid
    !   NDIMS - number of Cartesian dimensions 
    !   DIMS - array specifying the number of nodes in each dimension
    ! Examples:
    !   MPI_Dims_create(6, 2, dims)  ->  (3,2)
    !   MPI_Dims_create(7, 2, dims)  ->  (7,1)
    call MPI_Dims_create(mpisize, 2, pdims, mpierror)

    ! determine my coordinates (x,y)
    px = pdims(1)
    py = pdims(2)
    rx = mod(mpirank, px)
    ry = mpirank / px

    ! determine my four neighbors
    north = (ry - 1) * px + rx; if( (ry - 1) < 0  ) north = MPI_PROC_NULL
    south = (ry + 1) * px + rx; if( (ry + 1) >= py) south = MPI_PROC_NULL
    west = ry * px + rx - 1;    if( (rx - 1) < 0  ) west  = MPI_PROC_NULL
    east = ry * px + rx + 1;    if( (rx + 1) >= px) east  = MPI_PROC_NULL

    ! decompose the domain   
    bx = n / px             ! block size in x
    by = n / py             ! block size in y
    offx = (rx * bx) + 1    ! offset in x
    offy = (ry * by) + 1    ! offset in y

    ! initialize heat sources
    sources = reshape( [ n/2,   n/2,        &
                         n/3,   n/3,        &
                         n*4/5, n*8/9 ],    &
              shape(sources), order=[2, 1])

    do i = 1, nsources      ! determine which sources are in my patch
        locx = sources(i, 1) - offx
        locy = sources(i, 2) - offy    
        if(locx >= 0 .and. locx <= bx .and. locy >= 0 .and. locy <= by) then
            locnsources = locnsources + 1
            locsources(locnsources, 1) = locx + 2
            locsources(locnsources, 2) = locy + 2
        endif
    enddo

    ! allocate communication buffers
    allocate(sendnorthgz(bx))   ! send buffers
    allocate(sendsouthgz(bx))
    allocate(recvnorthgz(bx))   ! receive buffers
    allocate(recvsouthgz(bx))
    ! allocate two work arrays
    allocate(aold(bx+2, by+2)); aold = 0.0   ! 1-wide halo zones!
    allocate(anew(bx+2, by+2)); anew = 0.0   ! 1-wide halo zones!

    ! laco principal das iteracoes
    do iters = 1, niters, 2

        ! --- anew <- stencil(aold) ---
        if(north /= MPI_PROC_NULL) then 
            sendnorthgz = aold(2, 2:bx+1)
            recvnorthgz = 0.0
            call MPI_IRecv(recvnorthgz, bx, MPI_DOUBLE_PRECISION, north,  &
                            mpitag, MPI_COMM_WORLD, recvrequest(1), mpierror)
            call MPI_ISend(sendnorthgz, bx, MPI_DOUBLE_PRECISION, north,  &
                            mpitag, MPI_COMM_WORLD, sendrequest(1), mpierror)
        endif   
        if(south /= MPI_PROC_NULL) then 
            sendsouthgz = aold(bx+1, 2:bx+1)
            recvsouthgz(:) = 0.0
            call MPI_IRecv(recvsouthgz, bx, MPI_DOUBLE_PRECISION, south,  &
                            mpitag, MPI_COMM_WORLD, recvrequest(2), mpierror)
            call MPI_ISend(sendsouthgz, bx, MPI_DOUBLE_PRECISION, south,  &
                            mpitag, MPI_COMM_WORLD, sendrequest(2), mpierror)
        endif    
        if(east /= MPI_PROC_NULL) then 
            call MPI_IRecv(aold(2:bx+1, bx+2), bx, MPI_DOUBLE_PRECISION, east, &
                            mpitag, MPI_COMM_WORLD, recvrequest(3), mpierror)
            call MPI_ISend(aold(2:bx+1, bx+1), bx, MPI_DOUBLE_PRECISION, east, &
                            mpitag, MPI_COMM_WORLD, sendrequest(3), mpierror)
        endif    
        if(west /= MPI_PROC_NULL) then 
            call MPI_IRecv(aold(2:bx+1, 1), bx, MPI_DOUBLE_PRECISION, west, &
                           mpitag, MPI_COMM_WORLD, recvrequest(4), mpierror)
            call MPI_ISend(aold(2:bx+1, 2), bx, MPI_DOUBLE_PRECISION, west, &
                           mpitag, MPI_COMM_WORLD, sendrequest(4), mpierror)
            endif
        if(north /= MPI_PROC_NULL) then 
            call MPI_Wait(recvrequest(1), MPI_STATUS_IGNORE, mpierror)
            call MPI_Wait(sendrequest(1), MPI_STATUS_IGNORE, mpierror)
            aold(1, 2:bx+1)=recvnorthgz
        endif
        if(south /= MPI_PROC_NULL) then 
            call MPI_Wait(recvrequest(2), MPI_STATUS_IGNORE, mpierror)
            call MPI_Wait(sendrequest(2), MPI_STATUS_IGNORE, mpierror)
            aold(bx+2, 2:bx+1)=recvsouthgz
        endif
        if(east /= MPI_PROC_NULL) then 
            call MPI_Wait(recvrequest(3), MPI_STATUS_IGNORE, mpierror)
            call MPI_Wait(sendrequest(3), MPI_STATUS_IGNORE, mpierror)
        endif
        if(west /= MPI_PROC_NULL) then 
            call MPI_Wait(recvrequest(4), MPI_STATUS_IGNORE, mpierror)
            call MPI_Wait(sendrequest(4), MPI_STATUS_IGNORE, mpierror)
        endif  

        ! update grid points
        do j = 2, by+1 
            do i = 2, bx+1
                anew(i, j) = aold(i, j)/2.0 + (aold(i-1, j) + aold(i+1, j) +  &
                             aold(i, j-1) + aold(i, j+1)) / 4.0 / 2.0
            enddo
        enddo

        ! adiciona calor a malha
        do i = 1, locnsources
            anew(locsources(i, 1), locsources(i, 2)) =   &
                anew(locsources(i, 1), locsources(i, 2)) + energy
        enddo

        ! --- aold <- stencil(anew) ---
        if(north /= MPI_PROC_NULL) then 
            sendnorthgz=anew(2, 2:bx+1)
            call MPI_IRecv(recvnorthgz, bx, MPI_DOUBLE_PRECISION, north, mpitag,  &
                            MPI_COMM_WORLD, recvrequest(1), mpierror)
            call MPI_ISend(sendnorthgz, bx, MPI_DOUBLE_PRECISION, north, mpitag,  &
                            MPI_COMM_WORLD, sendrequest(1), mpierror)
        endif
        if(south /= MPI_PROC_NULL) then 
            sendsouthgz=anew(bx+1, 2:bx+1)
            call MPI_IRecv(recvsouthgz, bx, MPI_DOUBLE_PRECISION, south, mpitag,  &
                            MPI_COMM_WORLD, recvrequest(2), mpierror)   
            call MPI_ISend(sendsouthgz, bx, MPI_DOUBLE_PRECISION, south, mpitag,  &
                            MPI_COMM_WORLD, sendrequest(2), mpierror)
        endif
        if(east /= MPI_PROC_NULL) then 
            call MPI_IRecv(anew(2:bx+1, bx+2), bx, MPI_DOUBLE_PRECISION, east,  &
                            mpitag, MPI_COMM_WORLD, recvrequest(3), mpierror)
            call MPI_ISend(anew(2:bx+1, bx+1), bx, MPI_DOUBLE_PRECISION, east,  &
                            mpitag, MPI_COMM_WORLD, sendrequest(3), mpierror)
        endif
        if(west /= MPI_PROC_NULL) then 
            call MPI_IRecv(anew(2:bx+1, 1), bx, MPI_DOUBLE_PRECISION, west, mpitag,  &
                            MPI_COMM_WORLD, recvrequest(4), mpierror)
            call MPI_ISend(anew(2:bx+1, 2), bx, MPI_DOUBLE_PRECISION, west, mpitag,  &
                            MPI_COMM_WORLD, sendrequest(4), mpierror)
        endif
        if(north /= MPI_PROC_NULL) then 
            call MPI_Wait(recvrequest(1), MPI_STATUS_IGNORE, mpierror)
            call MPI_Wait(sendrequest(1), MPI_STATUS_IGNORE, mpierror)
            anew(1, 2:bx+1)=recvnorthgz
        endif
        if(south /= MPI_PROC_NULL) then 
            call MPI_Wait(recvrequest(2), MPI_STATUS_IGNORE, mpierror)
            call MPI_Wait(sendrequest(2), MPI_STATUS_IGNORE, mpierror)
            anew(bx+2, 2:bx+1)=recvsouthgz
        endif
        if(east /= MPI_PROC_NULL) then 
            call MPI_Wait(recvrequest(3), MPI_STATUS_IGNORE, mpierror)
            call MPI_Wait(sendrequest(3), MPI_STATUS_IGNORE, mpierror)
        endif
        if(west /= MPI_PROC_NULL) then 
            call MPI_Wait(recvrequest(4), MPI_STATUS_IGNORE, mpierror)
            call MPI_Wait(sendrequest(4), MPI_STATUS_IGNORE, mpierror)
        endif

        ! update grid points
        do j = 2, by+1 
            do i = 2, bx+1
                aold(i, j) = anew(i, j)/2.0 + (anew(i-1, j) + anew(i+1, j) +  &
                             anew(i, j-1) + anew(i, j+1)) / 4.0 / 2.0
            enddo
        enddo

        ! adiciona calor a malha:
        do i = 1, locnsources
            aold(locsources(i, 1), locsources(i, 2)) =  &
                aold(locsources(i, 1), locsources(i, 2)) + energy
        enddo

    enddo
   
    ! ALL REDUCE:
    heat = 0.0
    do j = 2, by+1 
        do i = 2, bx+1
            heat = heat + aold(i, j)
        enddo
    enddo
    call MPI_Allreduce(heat, rheat, 1, MPI_DOUBLE_PRECISION, MPI_SUM,  &
                       MPI_COMM_WORLD, mpierror)

    if(mpirank == 0) then
        mpiwtime = mpiwtime + MPI_Wtime()
        write(*, "('Heat='     f0.2' | ')", advance="no") rheat
        write(*, "('Time='    f0.4' | ')", advance="no") mpiwtime
        write(*, "('MPI_Size=' i0  ' | ')", advance="no") mpisize
        write(*, "('MPI_Dims=('i0','i0') | ')", advance="no") pdims
        write(*, "('bx,by=('i0','i0')')") bx,by
    endif

    call MPI_Finalize(mpierror)
end

Overwriting stencil_mpi.f90


In [3]:
%%bash
module load openmpi/gnu/4.0.1
mpif90  -O3  -o stencil_mpi  stencil_mpi.f90
ls -lh stencil_mpi

-rwxr-xr-x 1 xxxx.xxxx ampemi 23K Dec  2 22:18 stencil_mpi


In [12]:
%%bash
a='stencil_mpi'
b='/stnc/Fortran'
s='/prj/ampemi/xxxx.xxxx'$b
d='/scratch/ampemi/xxxx.xxxx'$b
cp $s/$a $d
ls -lh $d/$a

total 769K
-rwxr-xr-x 1 xxxx.xxxx ampemi 817K Nov  8 21:12 stencil_mpi
-rwxr-xr-x 1 xxxx.xxxx ampemi 765K Nov  8 18:59 stencil_seq


In [33]:
%%writefile stfopagnu485_16.srm
#!/bin/bash
#SBATCH --ntasks=16            # Total tasks(CPUs)
#SBATCH -p cpu_small           # Select partition
#SBATCH -J stcforpa            # Job name
#SBATCH --time=00:02:00        # Limit execution time

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tasks per node:' $SLURM_NTASKS_PER_NODE
echo '- Number of nodes:' $SLURM_JOB_NUM_NODES
echo '- Total tasks:' $SLURM_NTASKS
echo '- Nodes alocated:' $SLURM_JOB_NODELIST
echo '- Directory where sbatch was called ($SLURM_SUBMIT_DIR):'
echo $SLURM_SUBMIT_DIR
cd $SLURM_SUBMIT_DIR
nodeset -e $SLURM_JOB_NODELIST

# Module
echo '-- modules ----------------------------'
echo 'module load openmpi/gnu/4.0.1'
module load openmpi/gnu/4.0.1

# Executable
EXEC="/scratch/ampemi/xxxx.xxxx/stnc/Fortran/stencil_mpi 4800 1 500"

# Run
echo '-- srun -------------------------------'
echo '$ srun --mpi=pmi2 -n' $SLURM_NTASKS $EXEC
srun --mpi=pmi2 -n $SLURM_NTASKS $EXEC
echo '-- END --------------------------------'

Overwriting stfopagnu485_16.srm


<hr style="height:10px;border-width:0;background-color:green">

First measurement:

In [44]:
%%bash
d='/scratch/ampemi/xxxx.xxxx/stnc/Fortran'
cat $d/slurm-781013.out  #01
cat $d/slurm-781014.out  #04
cat $d/slurm-781015.out  #09
cat $d/slurm-781031.out  #16
cat $d/slurm-781017.out  #36
cat $d/slurm-781018.out  #49
cat $d/slurm-781019.out  #64
cat $d/slurm-781020.out  #81

- Job ID: 781013
- Tasks per node:
- Number of nodes: 1
- Total tasks: 1
- Nodes alocated: sdumont1429
- Directory where sbatch was called ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Fortran
sdumont1429
-- modules ----------------------------
module load openmpi/gnu/4.0.1
-- srun -------------------------------
$ srun --mpi=pmi2 -n 1 /scratch/ampemi/xxxx.xxxx/stnc/Fortran/stencil_mpi 4800 1 500
Heat=1500.00 | Time=21.8917 | MPI_Size=1 | MPI_Dims=(1,1) | bx,by=(4800,4800)
-- END --------------------------------
- Job ID: 781014
- Tasks per node:
- Number of nodes: 1
- Total tasks: 4
- Nodes alocated: sdumont1454
- Directory where sbatch was called ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Fortran
sdumont1454
-- modules ----------------------------
module load openmpi/gnu/4.0.1
-- srun -------------------------------
$ srun --mpi=pmi2 -n 4 /scratch/ampemi/xxxx.xxxx/stnc/Fortran/stencil_mpi 4800 1 500
Heat=1500.00 | Time=7.3403 | MPI_Size=4 | MPI_Dims=(2,2) | bx,by=(2400,2400)
-

<hr style="height:10px;border-width:0;background-color:green">

Second measurement:

In [2]:
%%bash
# 1x1=1, 2x2=4, 3x3=9, 4x4=16, 6x6=36, 7x7=49, 8x8=64, 9x9=81
sbatch stfopagnu485_01.srm
sbatch stfopagnu485_04.srm
sbatch stfopagnu485_09.srm
sbatch stfopagnu485_16.srm
sbatch stfopagnu485_36.srm
sbatch stfopagnu485_49.srm
sbatch stfopagnu485_64.srm
sbatch stfopagnu485_81.srm

Submitted batch job 788003
Submitted batch job 788004
Submitted batch job 788005
Submitted batch job 788006
Submitted batch job 788007
Submitted batch job 788008
Submitted batch job 788009
Submitted batch job 788010


In [3]:
! squeue -n stcforpa

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            788003 cpu_small stcforpa xxxx. PD       0:00      1 (Priority)
            788004 cpu_small stcforpa xxxx. PD       0:00      1 (Priority)
            788005 cpu_small stcforpa xxxx. PD       0:00      1 (Priority)
            788006 cpu_small stcforpa xxxx. PD       0:00      1 (Priority)
            788007 cpu_small stcforpa xxxx. PD       0:00      2 (Priority)
            788008 cpu_small stcforpa xxxx. PD       0:00      3 (Priority)
            788009 cpu_small stcforpa xxxx. PD       0:00      3 (Priority)
            788010 cpu_small stcforpa xxxx. PD       0:00      4 (Priority)


In [3]:
%%bash
d='/scratch/ampemi/xxxx.xxxx/stnc/Fortran'
cat $d/slurm-788003.out  #01
cat $d/slurm-788004.out  #04
cat $d/slurm-788005.out  #09
cat $d/slurm-788006.out  #16
cat $d/slurm-788007.out  #36
cat $d/slurm-788008.out  #49
cat $d/slurm-788009.out  #64
cat $d/slurm-788010.out  #81

- Job ID: 788003
- Tasks per node:
- Number of nodes: 1
- Total tasks: 1
- Nodes alocated: sdumont1149
- Directory where sbatch was called ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Fortran
sdumont1149
-- modules ----------------------------
module load openmpi/gnu/4.0.1
-- srun -------------------------------
$ srun --mpi=pmi2 -n 1 /scratch/ampemi/xxxx.xxxx/stnc/Fortran/stencil_mpi 4800 1 500
Heat=1500.00 | Time=21.9799 | MPI_Size=1 | MPI_Dims=(1,1) | bx,by=(4800,4800)
-- END --------------------------------
- Job ID: 788004
- Tasks per node:
- Number of nodes: 1
- Total tasks: 4
- Nodes alocated: sdumont1149
- Directory where sbatch was called ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Fortran
sdumont1149
-- modules ----------------------------
module load openmpi/gnu/4.0.1
-- srun -------------------------------
$ srun --mpi=pmi2 -n 4 /scratch/ampemi/xxxx.xxxx/stnc/Fortran/stencil_mpi 4800 1 500
Heat=1500.00 | Time=7.3660 | MPI_Size=4 | MPI_Dims=(2,2) | bx,by=(2400,2400)
-

<hr style="height:10px;border-width:0;background-color:green">

Third measurement:

In [5]:
%%bash
# 1x1=1, 2x2=4, 3x3=9, 4x4=16, 6x6=36, 7x7=49, 8x8=64, 9x9=81
sbatch stfopagnu485_01.srm
sbatch stfopagnu485_04.srm
sbatch stfopagnu485_09.srm
sbatch stfopagnu485_16.srm
sbatch stfopagnu485_36.srm
sbatch stfopagnu485_49.srm
sbatch stfopagnu485_64.srm
sbatch stfopagnu485_81.srm

Submitted batch job 788013
Submitted batch job 788014
Submitted batch job 788015
Submitted batch job 788016
Submitted batch job 788017
Submitted batch job 788018
Submitted batch job 788019
Submitted batch job 788020


In [6]:
! squeue -n stcforpa

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            788009 cpu_small stcforpa xxxx. PD       0:00      3 (Resources)
            788010 cpu_small stcforpa xxxx. PD       0:00      4 (Resources)
            788007 cpu_small stcforpa xxxx. PD       0:00      2 (Resources)
            788008 cpu_small stcforpa xxxx. PD       0:00      3 (Resources)
            788006 cpu_small stcforpa xxxx. PD       0:00      1 (Resources)
            788005 cpu_small stcforpa xxxx. PD       0:00      1 (Resources)
            788004 cpu_small stcforpa xxxx. PD       0:00      1 (Resources)
            788003 cpu_small stcforpa xxxx. PD       0:00      1 (Resources)
            788013 cpu_small stcforpa xxxx. PD       0:00      1 (Priority)
            788014 cpu_small stcforpa xxxx. PD       0:00      1 (Priority)
            788015 cpu_small stcforpa xxxx. PD       0:00      1 (Priority)
            788016 cpu_small stcforpa xxxx. PD       0:00      1 (Prior

Output:

In [4]:
%%bash
d='/scratch/ampemi/xxxx.xxxx/stnc/Fortran'
cat $d/slurm-788013.out  #01
cat $d/slurm-788014.out  #04
cat $d/slurm-788015.out  #09
cat $d/slurm-788016.out  #16
cat $d/slurm-788017.out  #36
cat $d/slurm-788018.out  #49
cat $d/slurm-788019.out  #64
cat $d/slurm-788020.out  #81

- Job ID: 788013
- Tasks per node:
- Number of nodes: 1
- Total tasks: 1
- Nodes alocated: sdumont1149
- Directory where sbatch was called ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Fortran
sdumont1149
-- modules ----------------------------
module load openmpi/gnu/4.0.1
-- srun -------------------------------
$ srun --mpi=pmi2 -n 1 /scratch/ampemi/xxxx.xxxx/stnc/Fortran/stencil_mpi 4800 1 500
Heat=1500.00 | Time=21.8653 | MPI_Size=1 | MPI_Dims=(1,1) | bx,by=(4800,4800)
-- END --------------------------------
- Job ID: 788014
- Tasks per node:
- Number of nodes: 1
- Total tasks: 4
- Nodes alocated: sdumont1149
- Directory where sbatch was called ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Fortran
sdumont1149
-- modules ----------------------------
module load openmpi/gnu/4.0.1
-- srun -------------------------------
$ srun --mpi=pmi2 -n 4 /scratch/ampemi/xxxx.xxxx/stnc/Fortran/stencil_mpi 4800 1 500
Heat=1500.00 | Time=7.3280 | MPI_Size=4 | MPI_Dims=(2,2) | bx,by=(2400,2400)
-

<hr style="height:10px;border-width:0;background-color:red">

## Version

In [4]:
%%bash
module load gcc/7.4
module load openmpi/gnu/4.0.1
mpif90 --version

GNU Fortran (GCC) 7.4.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
%%bash
module load openmpi/gnu/4.0.1
gcc --version
mpif90 --version

gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-36)
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

GNU Fortran (GCC) 4.8.5 20150623 (Red Hat 4.8.5-36)
Copyright (C) 2015 Free Software Foundation, Inc.

GNU Fortran comes with NO WARRANTY, to the extent permitted by law.
You may redistribute copies of GNU Fortran
under the terms of the GNU General Public License.
For more information about these matters, see the file named COPYING

